## Create spark context

In [1]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.config("spark.sql.shuffle.partitions", "2").appName("Analysis").master("local[2]").getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

/home/sunbeam/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [2]:
%run "../includes/configuration"

## Define schema

In [3]:
results_schema = StructType(fields=[
    StructField("resultId", IntegerType(), False),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("grid", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("positionText", StringType(), True),
    StructField("positionOrder", IntegerType(), True),
    StructField("points", FloatType(), True),
    StructField("laps", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
    StructField("fastestLap", IntegerType(), True),
    StructField("rank", IntegerType(), True),
    StructField("fastestLapTime", StringType(), True),
    StructField("fastestLapSpeed", FloatType(), True),
    StructField("statusId", StringType(), True),
])

## Read the CSV file from HDFS & apply schema

In [4]:
results_df = spark.read.schema(results_schema).csv(f"{data}/results.csv", header = True)

## Rename columns

In [5]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id")\
.withColumnRenamed("raceId", "race_id").withColumnRenamed("driverId", "driver_id")\
.withColumnRenamed("constructorId", "constructor_id").withColumnRenamed("positionText", "position_text")\
.withColumnRenamed("positionOrder", "position_order").withColumnRenamed("fastestLap", "fastest_lap")\
.withColumnRenamed("fastestLapTime", "fastest_lap_time").withColumnRenamed("fastestLapSpeed", "fastest_lap_speed")\

In [6]:
results_with_columns_df.columns

['result_id',
 'race_id',
 'driver_id',
 'constructor_id',
 'number',
 'grid',
 'position',
 'position_text',
 'position_order',
 'points',
 'laps',
 'time',
 'milliseconds',
 'fastest_lap',
 'rank',
 'fastest_lap_time',
 'fastest_lap_speed',
 'statusId']

## Drop unwanted columns

In [7]:
results_final_df = results_with_columns_df.drop(col("statusId"))

## Write outut to parquet file

In [8]:
results_final_df.write.mode('overwrite').parquet(f"{processed_folder_path}/results")